In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from tqdm import tqdm
from matplotlib import pyplot as plt

In [2]:
class DigitSumDataset(Dataset):
    def __init__(self, image_path, label_path):
        self.images = np.load(image_path)
        self.labels = np.load(label_path)
        self.images = torch.tensor(self.images, dtype=torch.float32) / 255.0
        self.labels = torch.tensor(self.labels, dtype=torch.float32)
        self.images = self.images.unsqueeze(1) 

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [3]:
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.resnet = models.resnet50(weights='IMAGENET1K_V1')
        self.resnet.conv1 = nn.Conv2d(
            1, 64, kernel_size=7, stride=2, padding=3, bias=False
        )

        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            torch.nn.Linear(num_features, 128),
            torch.nn.ReLU()
        )

        self.fc = nn.Sequential(
            torch.nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.resnet(x)
        x = self.fc(x)
        return x

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_dataset = DigitSumDataset("DL-project/data2.npy", "DL-Project/lab2.npy")

In [5]:
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
model = ResNetModel().to(device)
model.load_state_dict(torch.load("best_model.pth"))

model.eval()
acc = 0
for i, (images, labels) in enumerate(tqdm(test_dataloader)):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    preds = outputs.round().flatten()
    acc += torch.sum(preds == labels).item()
acc /= len(test_dataset)
print(f"Accuracy: {acc*100} %")

C:\Users\Kapil\AppData\Local\Temp\ipykernel_25192\3725644507.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))
100%|███

Accuracy: 0.4177
